## Part 4 Transition state

Supervisor: Prof. Aschauer

Assistants: Dr. Xing Wang

Room:	N216


### 1. Introduction

We now calculate the barrier for NH$_3$ dissociation. Before the real calculation, We will consider a simple example of the barrier for NH$_3$ inversion. We have to create an NH$_3$ molecule in the initial and inverted state (these have exactly the same energy), and then interpolate a band of images. Then, we use the `NEBEspresso` module to compute the barrier to inversion.

### 2. NH$_3$ inversion
initial and final states

In [5]:
from ase.structure import molecule
import numpy as np
from xespresso import Espresso
from ase.constraints import FixAtoms
from x3dase.visualize import view_x3d_n


atoms = molecule('NH3')
atoms.set_cell((10, 10, 10), scale_atoms=False)
constraint = FixAtoms(mask=[atom.symbol == 'N' for atom in atoms])
atoms.set_constraint(constraint)
# move N to origin
atoms.translate(-atoms.positions[0])
initial = atoms.copy()
view_x3d_n(final, output = 'htmls/nh3-inital.html')


In [7]:
final = atoms.copy()
final.positions[1:4, :] *= np.array([1, 1, -1])
view_x3d_n(final, output = 'htmls/nh3-final.html')

#### relax calculation

In [ ]:
from xespresso import Espresso
from xespresso.tools import mypool, fix_layers, dipole_correction

jobs = {
'initial': initial,
'final': final,
}
queue = {'ntasks': 10, 'partition': 'debug', 'time': '00:10:00'}
print('States    Energy (ev)')
for job, atoms in jobs.items():
    calc = Espresso(label = 'molecule/neb/nh3/{0}/{0}'.format(job),
                    pseudopotentials = mypseudo,
                    calculation = 'relax',
                    ecutwfc = 30,
                    kpts = (1, 1, 1),
                    queue = queue,
                   )
    atoms.calc = calc
    atoms.get_potential_energy()
    calc.read_results()
    jobs[job] = calc.results['atoms']
    e = calc.results['energy']
    print('{0:10s}  {1:10.3f}'.format(job, e))


#### NEB calculation

In [ ]:

from xespresso.neb import NEBEspresso

images = [jobs['initial'], jobs['final']]
path_data = {
'string_method' : 'neb',
'nstep_path': 50,
'opt_scheme': "broyden",
'num_of_images' : 7,
'k_max' : 0.3E0,
'k_min' : 0.2E0,
'CI_scheme' : "auto",
'path_thr': 0.1E0,
}
queue = {'nodes': 2, 'ntasks-per-node': 20, 'partition': 'debug', 'time': '00:10:00'}
calc = NEBEspresso(pseudopotentials = pseudopotentials, 
                 label  = 'molecule/neb/nh3/images',
                 package = 'neb',
                 queue = queue,
                 parallel = '-ni 5',
                 images = images,
                 path_data = path_data,
                 ecutwfc = 30,
                 kpts = (1, 1, 1),
                  )
#
calc.calculate()
calc.read_results()
calc.plot()


### 3. NH$_3$ dissociation
Please read the inital and final structure from your previous calculations, and do the NEB calculation. Please discuss with me when you are doing this.